<a href="https://colab.research.google.com/github/mattsankner/Test-Case-Creator-GPT-3.5/blob/main/Test_Case_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an app created during an internship at Insulet Corporation for purpose of demonstrating use of GPT/AI in increasing test coverage for an app.

Topic: an app that controls delivery and management of insulin to patients with diabetes through a proprietary device worn on the body.

The application revolves around a dataset that outlines specific requirements (or SRS) for the app, corresponding screens, and test cases. By inputting a query related to a specific requirement, the application, through GPT-3.5 Turbo, can generate detailed information about that requirement, including its associated test case. Furthermore, it's equipped to generate a new, unique test case corresponding to the same requirement, ensuring multifaceted test coverage.

Information used is sample public dummy data and is not proprietary/private company data.

We start by installing, importing, and initializing OpenAI API / GPT 3.5.

In [ ]:
!pip install --upgrade openai

In [ ]:
import openai
GPT_MODEL = "gpt-3.5-turbo"
openai.api_key="YOUR_OPEN_AI_API_KEY_HERE"

First, let's see if GPT 3.5 will make up a test case from a given requirement (unspecified SRS 003), an explanation of the app, acronyms, terms, etc. This is done so we can have a starting point to before we attach our data.

In [ ]:
query = 'Name a test case from SRS 003.'

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': '''
            You are an expert AI assistant who generates new and unique test cases for the
            First Time Setup (FTS) of an android app. The app is a mobile app that helps diabetics control
            their insulin doses. You have based your test case generation on an extensive knowledge of android
            mobile technology and knowledge of how each requirement corresponds to current test cases and different screens and their functionalities.

            Requirements are also called SRS, as in SRS 002. Test cases are referred to as TC (ex: TC004). TC’s can have multiple steps. Screens can
            have multiple options (actions on the screen).

            Use the following pieces of context to answer the question at the end.
            If you don't know the answer, just say you don't know. DO NOT try to make up an answer.
            If the question is not related to the context, politely respond that you are tuned to
            only answer questions that are related to the context.

            Some terms:
            Term, Acronym, Definition
            “Basal”, “ “, “The distribution of insulin administered to a diabetic over a normal day, excluding meals”
            “Bolus”, “ “, “Insulin distributed to a diabetic because the spike in blood sugar that results from a meal the diabetic has eaten or will eat”
            “Continuous Glucose Monitor”, “CGM”, “Device that monitors your blood glucose levels in order to recommend a given insulin dosage”
            “Personal Device Manager”, “PDM”, “Referring to the app we are testing as a whole”
            “Direction of Insulin Action”, “DIA”, “Length of time that insulin remains active and available in your body after a correction or a meal bolus”
         '''},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

I'm sorry, but I don't have access to the specific content of SRS 003. Could you please provide more information or specify a different SRS document?


We can see that it accepts that it doesn't know what requirement 003 is, so it asks for more information. Now, we will give it that information. Hard coded below is a csv database (hardcoded for illustration purposes; normally would use a loader with the LangChain library), that names 25 requirements, their test cases, and their screens. These are real terms and concepts, but fake data.

In [ ]:
requirements_data = '''
Requirement_ID,Requirement_Text,Test_Case_ID,Test_Case_Summary,Test_Case_Steps,Test_Case_Expected_Results,Screen_ID, Screen_Options

SRS 001, "User Authentication Requirement", TC003, "Verify Sign In Screen Functionality", "Step 1: Tap on Continue from the Welcome Screen; Step 2: Fill in the user and password fields; Step 3: Toggle the visibility of the password.", "The Sign In screen is displayed with user and password fields, a visible/hidden password button, a disabled Sign In button, and a 'Need Help?' button; Sign In button gets enabled; Password visibility toggles.", Screen 3, "Enter User and Password; Toggle Password visibility"
SRS 002, "Personalization Requirement", TC005, "Verify Personalize Lock Screen Message Screen", "Step 1: Tap on Continue from the 'All Set' screen; Step 2: Enter a lock screen message.", "Personalize Lock Screen Message screen is displayed with a keyboard, Back button, and Continue button; Keyboard pops up; App saves Lock Screen message; App moves to next screen; App returns to previous screen.", Screen 5, "Click inside text box; Enter Lock Screen message in text box; Click Continue Button; Click Back Button"
SRS 003, "Basal Setup Requirement", TC010, "Verify the Review Basal Program Functionality", "Step 1: From the Basal Window screen, after setting the start and end times and selecting basal rate, tap on Continue; Step 2: Review and tap on the Edit option; Step 3: Tap on Continue without making changes.", "Review Basal Program popup is displayed; App returns to Basal Window screen; App advances to the Review screen with an option to edit.", Screen 30, "Option to edit for each segment; Click Next Button; Click Cancel Button"
SRS 004, "Background Selection Requirement", TC006, "Verify the Select Background Screen Functionality", "Step 1: Tap on Continue from the Personalize Lock Screen Message screen; Step 2: Select a background; Step 3: Tap on the Continue Button", "Select Background screen is displayed with a Continue button; Selected background is highlighted; App moves to the next screen", Screen 6, "Select Background; Click Continue Button"
SRS 005, "PIN Setup Requirement", TC007, "Verify the Protect Your Insulin Device Screen", "Step 1: After the Select Background screen, Protect Your Insulin Device screen is displayed; Step 2: Enter a 4-digit PIN; Step 3: Tap to proceed", "Screen prompts to enter a 4-digit PIN, with a number keyboard and option to hide numbers; Each entry is replaced with asterisks, unless option to hide numbers is toggled off; App advances to the Re-enter Insulin Device screen", Screen 7, "Click on empty number box; set 1 digit of the 4-digit pin; Complete the 4-digit pin; Toggle PIN visibility; Click Continue Button; Click Back Button"
SRS 006, "Notification Acknowledgment Requirement", TC008, "Verify the 'Understand that I Will Receive Notifications' Screen", "Step 1: After Re-entering the Insulin Device PIN, reach the Notification Understanding screen; Step 2: Acknowledge the Notification Warning; Step 3: Tap on Continue", "Screen prompts to acknowledge receiving notifications and not turning them off; The Continue button becomes active; App advances to the next screen", Screen 9, "Click 'I Understand' Button; Click Back Button"
SRS 007, "Management Profile Completion Requirement", TC011, "Check the functionality of Temp Basal slider", "Step 1: From the Review screen, proceed to the Temp Basal slider screen; Step 2: Toggle the Temp Basal slider; Step 3: Toggle the Temp Basal slider back", "Screen provides option to toggle Temp Basal slider; A message pops up indicating the change in settings; Message disappears, and app returns to the original settings", Screen 11, "Continue button"
SRS 008, "Bolus Description Requirement", TC015, "Verify the Bolus Screen Description and Image", "Step 1: Tap on Continue from the Setup Bolus screen; Step 2: Observe the Bolus description and image", "App moves to the next screen; App displays the Bolus description and image", Screen 20, "Description and image of Bolus display"
SRS 009, "Target BG Setup Requirement", TC018, "Set up Target BG and Correct Above Values", "Step 1: Tap on Continue from the Set up Bolus screen; Step 2: Read the explanation; Step 3: Tap on Continue", "App moves to the next screen; App displays explanation for setting up target blood glucose (BG) and correction values", Screen 21, "Explanation, Click Continue Button; Click Back Button
SRS 010, "Bolus Setup Requirement", TC021, "Set up Bolus Screen", "Step 1: Adjust Start and End Time using sliders; Step 2: Adjust Target BG using slider (110-150 mg/dL); Step 3: Adjust Target Above using slider (target BG to 200 mg/dL)", "User sets start and end time; User sets target BG, and user sets target above values for each segment", Screen 22, "Adjust Start and End Time using sliders; Adjust Target BG using slider (110-150 mg/dL); Adjust Target Above using slider (target BG to 200 mg/dL)"
SRS 011, "Review Bolus Settings Requirement", TC023, "Review Target BG and Correct Above Values Entered for Each Time Segment", "Step 1: Display the list of time sequences with subsequent information; Step 2: Save/Cancel", "Display the list of time sequences with subsequent information; Save/Cancel options available", Screen 23, "Display the list of time sequences with subsequent info and edit button; Save/Cancel"
SRS 012, "Insulin to Carb Ratios Explanation Requirement", "", TC024, "Insulin to Carb Ratios Explanation Screen", "Step 1: Click Next/Cancel", "Proceed to the next screen; cancel the operation", Screen 24, "Next/Cancel"
SRS 013, "Insulin Dose Adjustment Requirement", TC027, "Setup Bolus - Segment #1", "Step 1: Adjust slider to decrease BG by a certain number in mg/dL with 1 unit of insulin; Step 2: Continue; Step 3: Back", "User sets the decrease in BG per 1 unit of insulin using the slider; User goes to next screen; User goes back", Screen 29, "Adjust slider to decrease BG by a certain number in mg/dL with 1 unit of insulin; Continue; Back"
SRS 014, "Insulin Action Duration Setup Requirement", TC031, "Set Duration of Insulin Action", "Step 1: Adjust slider to set the duration of insulin action in hours; Step 2: Continue; Step 3: Back", "User sets the duration of insulin action using the slider; User goes to next screen; User goes back", Screen 31, "Duration of insulin action in slider -> hrs; Continue; Back"
SRS 015, "Maximum Bolus Setup Requirement", TC032, "Set Maximum Bolus", "Step 1: Enter maximum bolus value in a text box using a number pad; Step 2: Continue; Step 3: Back", "User enters the maximum bolus value using a number pad; App moves to the next screen; App returns to the previous screen", Screen 32, "Enter maximum bolus value in a text box using a number pad; Click Continue Button; Click Back Button"
SRS 016, "Extended Bolus Setup Requirement", TC033, "Allow Extended Bolus", "Step 1: Toggle button to enable or disable extended bolus; Step 2: Continue; Step 3: Back", "User enables or disables extended bolus using a toggle button; App moves to the next screen; App returns to the previous screen", Screen 33, "Toggle button to enable or disable extended bolus; Click Continue Button; Click Back Button"
SRS 017, "First-time Setup Completion Requirement", TC034, "Welcome Screen (Confirmation that first-time setup is complete)", "Step 1: Confirmation that first-time setup is complete; Step 2: Option to set up a new device; Step 3: Option to defer setup of new device", "User is presented with confirmation that first-time setup is complete; User has options to set up a new device or defer setup of a new device", Screen 34, "Confirmation that first-time setup is complete; Option to set up a new device; Option to defer setup of new device"
SRS 018, "Review Screen Navigation Requirement", TC011, "Check the functionality of Temp Basal slider", "Step 1: From the Review screen, proceed to the Temp Basal slider screen; Step 2: Toggle the Temp Basal slider; Step 3: Toggle the Temp Basal slider back", "Screen provides option to toggle Temp Basal slider; A message pops up indicating the change in settings; Message disappears, and app returns to the original settings", Screen 11, "Continue button"
SRS 019, "Temporary Basal Rate Explanation Requirement", "The Check the functionality of Temp Basal slider test case shall demonstrate the functionality of the temporary basal rate feature. The app shall display an explanation of temporary basal rates on the screen.", TC011, "Check the functionality of Temp Basal slider", "Step 1: From the Review screen, proceed to the Temp Basal slider screen; Step 2: Toggle the Temp Basal slider; Step 3: Toggle the Temp Basal slider back", "Screen provides option to toggle Temp Basal slider; A message pops up indicating the change in settings; Message disappears, and app returns to the original settings", Screen 11, "Continue button"
SRS 020, "Bolus Confirmation Requirement", "The Set up Bolus Screen test case shall demonstrate the setup of bolus parameters. The app shall display a confirmation message after the user has adjusted the start and end time and target BG and target above values.", TC021, "Set up Bolus Screen", "Step 1: Adjust Start and End Time using sliders; Step 2: Adjust Target BG using slider (110-150 mg/dL); Step 3: Adjust Target Above using slider (target BG to 200 mg/dL)", "User sets start and end time; User sets target BG, and user sets target above values for each segment", Screen 22, "Adjust Start and End Time using sliders; Adjust Target BG using slider (110-150 mg/dL); Adjust Target Above using slider (target BG to 200 mg/dL)"
SRS 021, "Insulin Action Review Requirement", "The app shall allow the user to review the insulin action duration settings. The Review Insulin Action Duration screen shall display the entered duration of insulin action in hours. The user shall have the option to edit the duration or proceed to the next screen.", TC 021, "Verify Insulin Action Review Screen Functionality", "Step 1: Navigate to the Review Insulin Action Duration screen; Step 2: Observe the displayed insulin action duration; Step 3: Tap on the Edit option; Step 4: Adjust the insulin action duration; Step 5: Tap on the Continue button; Step 6: Tap on the Back button", "The Review Insulin Action Duration screen displays the entered insulin action duration; The user is able to edit the insulin action duration; The user proceeds to the next screen; The user returns to the previous screen", Screen 21, "Option to edit the insulin action duration; Continue button; Back button"
SRS 022, "Maximum Bolus Review Requirement", "The app shall allow the user to review the maximum bolus value. The Review Maximum Bolus screen shall display the entered maximum bolus value. The user shall have the option to edit the value or proceed to the next screen.", TC 022, "Verify Maximum Bolus Review Screen Functionality", "Step 1: Navigate to the Review Maximum Bolus screen; Step 2: Observe the displayed maximum bolus value; Step 3: Tap on the Edit option; Step 4: Adjust the maximum bolus value; Step 5: Tap on the Continue button; Step 6: Tap on the Back button", "The Review Maximum Bolus screen displays the entered maximum bolus value; The user is able to edit the maximum bolus value; the user proceeds to the next screen; The user returns to the previous screen.\'94, Screen 32, "Maximum Bolus Review Screen", "Edit the maximum bolus value; Continue button; Back button"
SRS 023, "Extended Bolus Review Requirement", "The app shall allow the user to review the extended bolus settings. The Review Extended Bolus screen shall display the status of the extended bolus (enabled or disabled). The user shall have the option to edit the settings or proceed to the next screen.", TC 023, "Verify Extended Bolus Review Screen Functionality", "Step 1: Navigate to the Review Extended Bolus screen; Step 2: Observe the displayed status of the extended bolus; Step 3: Tap on the Edit option; Step 4: Adjust the extended bolus settings; Step 5: Tap on the Continue button; Step 6: Tap on the Back button", "The Review Extended Bolus screen displays the entered status of the extended bolus; The user is able to edit the extended bolus settings; The user proceeds to the next screen; The user returns to the previous screen" Screen 33, "Display status of extended bolus (enabled or disabled); Edit Settings; Continue button; Back button"
SRS 024, "Welcome Screen Navigation Requirement", TC 002, "Verify Welcome Screen Navigation", "Step 1: Navigate to the Welcome screen; Step 2: Tap on the 'Set Up New Device' button; Step 3: Tap on the 'Defer Setup' button", "The user is presented with the option to set up a new device; The user is presented with the option to defer the setup; The user proceeds to the respective screens based on the selected option.\'94, Screen 34, "Option to set up a new device; Option to defer setup of new device"
SRS 025, "Basal Segment Editing Requirement", TC 010, "Verify Basal Segment Editing Functionality", "Step 1: Navigate to the Review Basal Program screen; Step 2: Observe the displayed time segments for basal delivery; Step 3: Tap on the Edit option; Step 4: Adjust the start and end time, as well as the basal rate for each segment; Step 5: Tap on the Continue button; Step 6: Tap on the Back button", "The Review Basal Program screen displays the entered time segments for basal delivery; The user is able to edit the start and end time, as well as the basal rate for each segment; The user proceeds to the next screen; the user returns to the previous screen", Screen 25, "Option to edit the start and end time, edit the basal rate for each segment; Continue button; Back button"
'''

Now, we will build a query that gives GPT 3.5 directions for what it should do with the information given to it. You can see requirements_data placed under the "Article:" string. It should describe the given requirements and one of its test cases, and then generate a new test case for the same requirement.

There is an additional condition specified for the purpose of seeing how detailed GPT 3.5 can be; we ask it to make sure one step in the newly generated test case is identical to a step in the first test case. The language is slightly ambiguous (again to see how smart it can be).

In [ ]:
query = f"""Use the database about the requirements for my app and each requirement's associated screens and test cases to answer the subsequent question. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{requirements_data}
\"\"\"

Question: Describe SRS 023. Write the details of one current test case,
And then generate a new, unique test case that corresopnds to the same
SRS and screen.
The test case should have at least one identical step to the one you identify first."""

Lastly, we will use an OpenAI Chat Completion to specify the system role for GPT 3.5. Our query is attached to the completion in the user role.

In [ ]:
response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': '''
            You are an expert AI assistant who generates new and unique test cases for the
            First Time Setup (FTS) of an android app. The app is a mobile app that helps diabetics control
            their insulin doses. You have based your test case generation on an extensive knowledge of android
            mobile technology and knowledge of how each requirement corresponds to current test cases and different screens and their functionalities.

            Requirements are also called SRS, as in SRS 002. Test cases are referred to as TC (ex: TC004). TC’s can have multiple steps. Screens can
            have multiple options (actions on the screen).

            “Continuous Glucose Monitor”, “CGM”
            “Personal Device Manager”, “PDM”, “Our app"
            “Direction of Insulin Action”, “DIA”, “Length of time that insulin remains active after meal bolus"

            '''},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)



Lastly, we print GPT 3.5's response.

In [ ]:
print(response['choices'][0]['message']['content'])

SRS 023 states the "Extended Bolus Review Requirement". This requirement specifies that the app should allow the user to review the extended bolus settings. The Review Extended Bolus screen should display the status of the extended bolus (enabled or disabled). The user should have the option to edit the settings or proceed to the next screen.

One current test case that corresponds to SRS 023 is TC023 - "Verify Extended Bolus Review Screen Functionality". This test case includes the following steps:
1. Navigate to the Review Extended Bolus screen.
2. Observe the displayed status of the extended bolus.
3. Tap on the Edit option.
4. Adjust the extended bolus settings.
5. Tap on the Continue button.
6. Tap on the Back button.

Now, let's generate a new, unique test case that corresponds to SRS 023 and the Review Extended Bolus screen:

Test Case: TC035 - "Verify Extended Bolus Status Display"
Summary: Verify that the Review Extended Bolus screen displays the current status of the extended

As you can see, it's extremely impressive what we can do with a simple GPT integration! A response like the one given could lessen developer time thinking of new test cases to perform.

To build off of this app, I would add a csv or pdf loader and quadruple the data, while also attaching android app documentation. This way, GPT 3.5 could refer to the android app documentation and pull from options on their phone to use in their test cases.

Example: let's say we haven't ran a test yet where a user turns on airplane mode while recieving insulin, and we haven't thought of that test case either. GPT 3.5 could then, with an adjusted system direction, read that android has such thing as "airplane mode," and suggest we test it in all of our app functions. It could then format it as a normal looking test case, with a new or existing requirement.